In [ ]:
from itertools import product
n = 4
symbols = [' ', 'X', 'O']
all_boards = list(product(symbols, repeat=n*n))  # Generate all 3^(n*n) combinations
assert (3**(n*n)) == len(all_boards)
all_valid_boards = []

for board in all_boards:
    x_count = board.count('X')
    o_count = board.count('O')
    
    # Valid boards must satisfy these conditions:
    if x_count == o_count or x_count == o_count + 1:
        all_valid_boards.append(board)

In [ ]:
print(len(all_boards))
print(len(all_valid_boards))

In [1]:
import numpy as np
from itertools import product
from collections import defaultdict

n = 3

transformations = [
    lambda x: x,                         # Identity
    lambda x: np.rot90(x, 1),            # Rotate 90°
    lambda x: np.rot90(x, 2),            # Rotate 180°
    lambda x: np.rot90(x, 3),            # Rotate 270°
    lambda x: np.fliplr(x),              # Horizontal reflection
    lambda x: np.flipud(x),              # Vertical reflection
    lambda x: np.transpose(x),           # Diagonal reflection (TL-BR)
    # lambda x: np.fliplr(np.transpose(x)) # Diagonal reflection (TR-BL)
]

inverse_transformations = [
    lambda x: x,                         # Identity
    lambda x: np.rot90(x, 3),            # Rotate 90° inverse (rotate 270°)
    lambda x: np.rot90(x, 2),            # Rotate 180° inverse
    lambda x: np.rot90(x, 1),            # Rotate 270° inverse (rotate 90°)
    lambda x: np.fliplr(x),              # Horizontal reflection inverse
    lambda x: np.flipud(x),              # Vertical reflection inverse
    lambda x: np.transpose(x),           # Diagonal reflection (TL-BR) inverse
    # lambda x: np.transpose(np.fliplr(x))# Diagonal reflection (TR-BL) inverse
]

original_actions = np.array(range(n * n)).reshape(n, n)
for i, transform in enumerate(transformations):
    assert inverse_transformations[i](transform(original_actions)).flatten().tolist() == original_actions.flatten().tolist()

transformed_actions = [transform(original_actions).flatten().tolist() for transform in transformations]
for i in range(len(transformed_actions)):
    for j in range(i + 1, len(transformed_actions)):
        assert transformed_actions[i] != transformed_actions[j]

def generate_all_valid_boards():
    symbols = [' ', 'X', 'O']
    all_boards = list(product(symbols, repeat=n*n))  # Generate all 3^(n*n) combinations
    assert (3**(n*n)) == len(all_boards)
    all_valid_boards = []

    for board in all_boards:
        x_count = board.count('X')
        o_count = board.count('O')
        
        # Valid boards must satisfy these conditions:
        if x_count == o_count or x_count == o_count + 1:
            all_valid_boards.append(board)

    return all_valid_boards

def board_to_matrix(board):
    return np.array(board).reshape(n, n)

def matrix_to_board(matrix):
    return matrix.flatten().tolist()

def generate_symmetries(board):
    matrix = board_to_matrix(board)
    symmetries = [transform(matrix) for transform in transformations]
    return [matrix_to_board(sym) for sym in symmetries]

def get_canonical_representation(board):
    symmetries = generate_symmetries(board)
    min_symmetry = min(symmetries)
    return tuple(min_symmetry), symmetries.index(min_symmetry)

# Generate all empty positions on the board
def get_empty_positions(board):
    return [i for i, cell in enumerate(board) if cell == ' ']

def display_board(board):
    print(f" {board[0]} | {board[1]} | {board[2]} ")
    print("---+---+---")
    print(f" {board[3]} | {board[4]} | {board[5]} ")
    print("---+---+---")
    print(f" {board[6]} | {board[7]} | {board[8]} ")
    print("\n")

def get_next_player(board):
    x_count = board.count('X')
    o_count = board.count('O')
    return 'X' if x_count == o_count else 'O'

def get_next_board(board, action):
    new_board = list(board)
    next_player = get_next_player(board)
    if next_player == 'X':
        new_board[action] = 'X'
    else:
        new_board[action] = 'O'
    
    return tuple(new_board)

# Generate all valid Tic-Tac-Toe boards
all_valid_boards = generate_all_valid_boards()
state_action_pairs = 0
for valid_board in all_valid_boards:
    empty_positions = get_empty_positions(valid_board)
    state_action_pairs += len(empty_positions)

# Generate all canonical Tic-Tac-Toe boards
all_canonical_boards = set()
get_canonical_boards = {}
get_transform = {}
get_inverse_transform = {}
get_canonical_actions = {}
get_inverse_canonical_actions = {}
for valid_board in all_valid_boards:
    canonical_board, transform_idx = get_canonical_representation(valid_board)
    all_canonical_boards.add(canonical_board)
    get_canonical_boards[valid_board] = canonical_board
    get_transform[valid_board] = transformations[transform_idx]
    get_inverse_transform[valid_board] = inverse_transformations[transform_idx]
    get_inverse_canonical_actions[valid_board] = get_transform[valid_board](original_actions).flatten().tolist()
    get_canonical_actions[valid_board] = get_inverse_transform[valid_board](original_actions).flatten().tolist()

all_canonical_boards = sorted(list(all_canonical_boards))
all_canonical_actions = {}
canonical_state_action_pairs = 0
for canonical_board in all_canonical_boards:
    empty_positions = get_empty_positions(canonical_board)
    all_canonical_actions[canonical_board] = empty_positions
    canonical_state_action_pairs += len(empty_positions)

def get_canonical_board(board):
    return get_canonical_boards[tuple(board)]

def get_canonical_action(board, action):
    return get_canonical_actions[board][action]

def get_inverse_canonical_action(board, canonical_action):
    return get_inverse_canonical_actions[board][canonical_action]

def canonicalize(board, action):
    canonical_board = get_canonical_board(board)
    canonical_action = get_canonical_action(board, action)
    return canonical_board, canonical_action

canonical_board_to_next_canonical_board = {}
all_next_canonical_boards = set()
for canonical_board in all_canonical_boards:
    canonical_actions_to_next_canonical_board = {}
    for canonical_action in all_canonical_actions[canonical_board]:
        next_board = get_next_board(canonical_board, canonical_action)
        next_canonical_board = get_canonical_board(next_board)
        all_next_canonical_boards.add(next_canonical_board)
        canonical_actions_to_next_canonical_board[canonical_action] = next_canonical_board

    canonical_board_to_next_canonical_board[canonical_board] = canonical_actions_to_next_canonical_board

qMatrix = defaultdict(lambda: -1)
for i, next_canonical_board in enumerate(all_next_canonical_boards):
    qMatrix[next_canonical_board] = i

def get(board=None, action=None):
    if board is None and action is None:
        return qMatrix
    if action is None and board is not None:
        actions = get_empty_positions(board)
        canonical_actions = [get_canonical_action(board, action) for action in actions]
        canonical_board = get_canonical_board(board)
        return [qMatrix[canonical_board_to_next_canonical_board[canonical_board][canonical_action]] for canonical_action in canonical_actions]
    if action is not None and board is not None:
        return qMatrix[canonical_board_to_next_canonical_board[get_canonical_board(board)][get_canonical_action(board, action)]]

def set(board, action, value):
    qMatrix[canonical_board_to_next_canonical_board[get_canonical_board(board)][get_canonical_action(board, action)]] = value

def displayQ(valid_board):
    valid_actions = get_empty_positions(valid_board)
    canonical_board = get_canonical_board(valid_board)
    Qvalid_board = list(valid_board)
    for valid_action in valid_actions:
        Qvalid_board[valid_action] = qMatrix[canonical_board_to_next_canonical_board[canonical_board][get_canonical_action(valid_board, valid_action)]]

    print("\n")
    print(np.array(Qvalid_board).reshape(n,n))

print(f"Number of boards:                               {(3**(n*n))}")
print(f"Number of state-action pairs:                   {(3**(n*n))*(n*n)}")
print(f"Number of valid boards:                         {len(all_valid_boards)}")    
print(f"Number of valid state-action pairs:             {state_action_pairs}")
print(f"Number of canonical boards:                     {len(all_canonical_boards)}")
print(f"Number of canonical state-action pairs:         {canonical_state_action_pairs}")
print(f"Number of next canonical boards:                {len(all_next_canonical_boards)}")
print(f"Number of reduced canonical state-action pairs: {len(all_next_canonical_boards)}")

board1 = ('X', ' ', ' ', ' ', 'O', ' ', ' ', ' ', ' ')
board2 = (' ', ' ', ' ', ' ', 'O', 'X', ' ', ' ', ' ')
displayQ(board1)
displayQ(board2)
assert get(board1, 5) == get(board2, 0)
print(f"{get(board1, 5)} == {get(board2, 0)}")

Number of boards:                               19683
Number of state-action pairs:                   177147
Number of valid boards:                         6046
Number of valid state-action pairs:             19107
Number of canonical boards:                     1520
Number of canonical state-action pairs:         4808
Number of next canonical boards:                1073
Number of reduced canonical state-action pairs: 1073


[['X' '141' '223']
 ['141' 'O' '230']
 ['223' '230' '445']]


[['230' '358' '141']
 ['644' 'O' 'X']
 ['230' '358' '141']]
230 == 230


Number of boards:                               19683
Number of state-action pairs:                   177147
Number of valid boards:                         6046
Number of valid state-action pairs:             19107
Number of canonical boards:                     1520
Number of canonical state-action pairs:         4808
Number of next canonical boards:                1073
Number of reduced canonical state-action pairs: 1073

In [ ]:
qMatrix = defaultdict(lambda: -1)
board = (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
set(board, 0, 0.2)
displayQ(board)

In [ ]:
valid_board = all_valid_boards[5]
actions = get_empty_positions(valid_board)
canonical_actions = [get_canonical_action(valid_board, action) for action in actions]
print(f"Board:             {valid_board}")
print(f"Canonical board:   {get_canonical_board(valid_board)}")
print(f"Actions:           {actions}")
print(f"Canonical actions: {sorted(canonical_actions)}")
print(f"Canonical actions: {sorted(all_canonical_actions[get_canonical_board(valid_board)])}")

tot = 0
for i, valid_board in enumerate(all_valid_boards):
    actions = get_empty_positions(valid_board)
    canonical_board = get_canonical_board(valid_board)
    canonical_actions1 = sorted(all_canonical_actions[canonical_board])
    canonical_actions2 = sorted([get_canonical_action(valid_board, action) for action in actions])
    if not canonical_actions2 == canonical_actions1:
        tot += 1

print(f"Number of wrong canonical actions:      {tot}/{len(all_valid_boards)}")

In [ ]:
for i in range(10):
    if i == 10 - 1:
        print(f"final i: {i}")
    else:
        print(f"{i}, {i + 1}")

for i in reversed(range(10)):
    if i == 10 - 1:
        print(f"final i: {i}")
    else:
        print(f"{i}, {i + 1}")

In [ ]:
import collections

# initializing deque
de = collections.deque([1, 2, 3], maxlen=3)
print("deque: ", de)

# using append() to insert element at right end
# inserts 4 at the end of deque
de.append(4)

# printing modified deque
print("\nThe deque after appending at right is : ")
print(de)

# using appendleft() to insert element at left end
# inserts 6 at the beginning of deque
de.appendleft(6)

# printing modified deque
print("\nThe deque after appending at left is : ")
print(de)

In [ ]:
# Python3 program to demonstrate
# the use of sample() function .

# import random 
import random


# Prints list of random items of
# length 3 from the given list.
list1 = [1, 2, 3, 4, 5, 6] 
print("With list:", random.sample(list1, 3))
print("With list:", random.sample(list1, 2))

# Prints list of random items of
# length 4 from the given string. 
string = "GeeksforGeeks"
print("With string:", random.sample(string, 4))

# Prints list of random items of
# length 4 from the given tuple.
tuple1 = ("ankit", "geeks", "computer", "science",
                   "portal", "scientist", "btech")
print("With tuple:", random.sample(tuple1, 4))

In [ ]:
class LazyComputeDict(dict):
    def __init__(self, compute_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.compute_func = compute_func
    
    def __getitem__(self, key):
        if key not in self:
            # Compute and store the value if it doesn't exist
            self[key] = self.compute_func(key)
        return super().__getitem__(key)

def compute_value(key):
    # Replace this with the actual computation logic
    return key ** 2

# Usage
my_dict = LazyComputeDict(lambda key : key ** 2)

# Accessing a key
key = 50
value = my_dict[key]  # Computes and stores the value on the fly
print(value)


def create_level2_lazy_dict(compute_func):
    def level1_compute(outer_key):
        # Create another LazyComputeDict for the second level
        return LazyComputeDict(lambda inner_key: compute_func(outer_key, inner_key))
    
    # Return the top-level LazyComputeDict
    return LazyComputeDict(level1_compute)

# Example computation function for level-2
def compute_value(outer_key, inner_key):
    return f"Computed for {outer_key}, {inner_key}"

# Create the level-2 LazyComputeDict
nested_dict = create_level2_lazy_dict(compute_value)

# Accessing values dynamically
print(nested_dict['group1']['item1'])  # Triggers computation for ('group1', 'item1')
print(nested_dict['group2']['item2'])  # Triggers computation for ('group2', 'item2')

# Accessing the entire structure
print(nested_dict)